<a href="https://colab.research.google.com/github/1804054Miraz/My_Thesis_Work/blob/main/Apply_ML_algorithm_in_EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import numpy as np
import pandas as pd
import statistics as st
import scipy.stats as scst
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
from scipy.stats import normaltest
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
path = '/content/drive/MyDrive/Dataset/EEG_Features_Full.csv'
EEG_Features = pd.read_csv(path)

In [ ]:
# Assuming 'EEG_Features' is your DataFrame
# Let's fill NaN values in each column with the mean of that column
EEG_Features.fillna(EEG_Features.mean(), inplace=True)


In [ ]:
# EEG_Features=pd.DataFrame(EEG_Features)
# EEG_Features.rename(columns={'1':'MAX', '2':'MIN', '3':'MEAN', '4':'STD', '5':'VAR','6': 'SKW', '7':'KUR','8': 'SE','9': 'LEE','10': 'PPA','11': 'label'}, inplace=True)

In [ ]:
# EEG_Features.head()

,Unnamed: 0,MAX,MIN,MEAN,STD,VAR,SKW,KUR,SE,LEE,PPA,label
0,0,42.826,-42.379,-2.133502,2.364078,878.795928,-0.972187,-0.039005,242.224337,486.902576,69.336,1.0
1,1,78.573,-74.107,-5.455840,4.331395,8026.370564,0.722541,1.547945,263.171864,479.856202,98.145,1.0
2,2,21.383,-67.413,-7.669460,2.320169,942.009385,-0.591528,-0.040461,256.546093,489.405860,78.125,1.0
3,3,79.356,-81.502,-5.391158,4.559711,9797.916632,0.429586,3.814154,269.395252,479.610238,100.098,1.0
4,4,26.835,-59.835,-4.580334,2.461220,1097.870546,0.081715,-0.353628,266.733494,489.061640,69.336,1.0


In [ ]:
# EEG_Features.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
EEG_Features

,MAX,MIN,MEAN,STD,VAR,SKW,KUR,SE,LEE,PPA,label
0,42.826,-42.379,-2.133502,2.364078,878.795928,-0.972187,-0.039005,242.224337,486.902576,69.336,1.0
1,78.573,-74.107,-5.455840,4.331395,8026.370564,0.722541,1.547945,263.171864,479.856202,98.145,1.0
2,21.383,-67.413,-7.669460,2.320169,942.009385,-0.591528,-0.040461,256.546093,489.405860,78.125,1.0
3,79.356,-81.502,-5.391158,4.559711,9797.916632,0.429586,3.814154,269.395252,479.610238,100.098,1.0
4,26.835,-59.835,-4.580334,2.461220,1097.870546,0.081715,-0.353628,266.733494,489.061640,69.336,1.0
...,...,...,...,...,...,...,...,...,...,...,...
7617,55.176,-99.548,-4.674924,6.046616,31778.109610,-0.088029,-1.168351,328.116727,487.646729,124.023,0.0
7618,42.084,-44.108,2.423221,2.903113,2427.901252,0.499885,-0.328437,299.485927,491.241884,69.336,0.0
7619,44.240,-66.406,0.690167,4.618951,14518.874109,0.239581,0.135848,317.144996,487.114770,89.355,0.0
7620,0.437,-0.051,0.000722,0.024173,0.000022,0.030898,1.541187,0.000000,509.672425,0.488,0.0


In [ ]:
X = EEG_Features.iloc[:,:-1]
y = EEG_Features.iloc[:,-1]
X

,MAX,MIN,MEAN,STD,VAR,SKW,KUR,SE,LEE,PPA
0,42.826,-42.379,-2.133502,2.364078,878.795928,-0.972187,-0.039005,242.224337,486.902576,69.336
1,78.573,-74.107,-5.455840,4.331395,8026.370564,0.722541,1.547945,263.171864,479.856202,98.145
2,21.383,-67.413,-7.669460,2.320169,942.009385,-0.591528,-0.040461,256.546093,489.405860,78.125
3,79.356,-81.502,-5.391158,4.559711,9797.916632,0.429586,3.814154,269.395252,479.610238,100.098
4,26.835,-59.835,-4.580334,2.461220,1097.870546,0.081715,-0.353628,266.733494,489.061640,69.336
...,...,...,...,...,...,...,...,...,...,...
7617,55.176,-99.548,-4.674924,6.046616,31778.109610,-0.088029,-1.168351,328.116727,487.646729,124.023
7618,42.084,-44.108,2.423221,2.903113,2427.901252,0.499885,-0.328437,299.485927,491.241884,69.336
7619,44.240,-66.406,0.690167,4.618951,14518.874109,0.239581,0.135848,317.144996,487.114770,89.355
7620,0.437,-0.051,0.000722,0.024173,0.000022,0.030898,1.541187,0.000000,509.672425,0.488


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.29, random_state=42)
classifier = LogisticRegression()

parameter = {'penalty':['l1','l2','elasticnet'], 'C':[1,2,3,4,5,6,10,20,30,40],'max_iter':[100,200,300]}
# parameter = {
#     'C': [0.1, 1, 10, 100],
#     'penalty': ['none', 'l2'],
#     'max_iter': [100, 200, 300]
# }

classifier_regressor = GridSearchCV(classifier, param_grid=parameter, scoring='accuracy', cv=5)


In [ ]:
%%capture
classifier_regressor.fit(X_train, y_train)

In [ ]:
print(classifier_regressor.best_params_)
print(classifier_regressor.best_score_)
y_pred = classifier_regressor.predict(X_test)

score = accuracy_score(y_test,y_pred)
print('Score = ',score)

{'C': 6, 'max_iter': 200, 'penalty': 'l2'}
0.6449813365010931
Score =  0.6910900045228403


In [ ]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.48      0.65      1525
         1.0       0.00      0.00      0.00         0

    accuracy                           0.48      1525
   macro avg       0.50      0.24      0.33      1525
weighted avg       1.00      0.48      0.65      1525



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#EDA
# sns.pairplot(EEG_Features, hue='label')

In [ ]:
# X = EEG_Features.drop('label', axis=1)
# y = EEG_Features['label']

# Assuming X contains your features and y contains your labels (alcoholic or non-alcoholic)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.61
Classification Report:
               precision    recall  f1-score   support

         0.0       0.60      0.56      0.58       927
         1.0       0.61      0.65      0.63       979

    accuracy                           0.61      1906
   macro avg       0.61      0.61      0.61      1906
weighted avg       0.61      0.61      0.61      1906



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd

# # Assuming EEG_Features is your DataFrame with features and labels
# # X contains features, and y contains labels (alcoholic or non-alcoholic)
# # Replace 'LabelColumn' with the actual column name containing your labels
# X = EEG_Features.drop('label', axis=1)
# y = EEG_Features['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the SVM model
# You can adjust the hyperparameters such as C and kernel based on your data
model = SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)

# Train the SVM model
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.72
Classification Report:
               precision    recall  f1-score   support

         0.0       0.71      0.72      0.71       737
         1.0       0.73      0.72      0.73       788

    accuracy                           0.72      1525
   macro avg       0.72      0.72      0.72      1525
weighted avg       0.72      0.72      0.72      1525



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model
# You can adjust hyperparameters such as n_estimators, max_depth, etc.
model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

# Train the Random Forest model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

         0.0       0.69      0.71      0.70       737
         1.0       0.72      0.71      0.72       788

    accuracy                           0.71      1525
   macro avg       0.71      0.71      0.71      1525
weighted avg       0.71      0.71      0.71      1525



In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
# You can adjust hyperparameters such as learning_rate, n_estimators, max_depth, etc.
model = XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=3, random_state=42)

# Train the XGBoost model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.72
Classification Report:
               precision    recall  f1-score   support

         0.0       0.70      0.74      0.72       737
         1.0       0.74      0.70      0.72       788

    accuracy                           0.72      1525
   macro avg       0.72      0.72      0.72      1525
weighted avg       0.72      0.72      0.72      1525



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the KNN model
# You can adjust hyperparameters such as n_neighbors, weights, etc.
model = KNeighborsClassifier(n_neighbors=5, weights='uniform')

# Train the KNN model
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.65
Classification Report:
               precision    recall  f1-score   support

         0.0       0.63      0.64      0.64       737
         1.0       0.66      0.65      0.65       788

    accuracy                           0.65      1525
   macro avg       0.65      0.65      0.65      1525
weighted avg       0.65      0.65      0.65      1525



In [ ]:
# Deep learning algorithm

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the FNN model
model = Sequential([
    Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the FNN
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the FNN on the test set
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"FNN Accuracy: {accuracy:.2f}")
print("FNN Classification Report:\n", classification_report(y_test, y_pred))


Epoch 1/10
153/153 [==============================] - 2s 6ms/step - loss: 0.6322 - accuracy: 0.6522 - val_loss: 0.6178 - val_accuracy: 0.6680
Epoch 2/10
153/153 [==============================] - 1s 4ms/step - loss: 0.6018 - accuracy: 0.6859 - val_loss: 0.5960 - val_accuracy: 0.6885
Epoch 3/10
153/153 [==============================] - 1s 5ms/step - loss: 0.5932 - accuracy: 0.6953 - val_loss: 0.5934 - val_accuracy: 0.6893
Epoch 4/10
153/153 [==============================] - 1s 4ms/step - loss: 0.5839 - accuracy: 0.7039 - val_loss: 0.5901 - val_accuracy: 0.6910
Epoch 5/10
153/153 [==============================] - 1s 4ms/step - loss: 0.5824 - accuracy: 0.7033 - val_loss: 0.5867 - val_accuracy: 0.6926
Epoch 6/10
153/153 [==============================] - 1s 5ms/step - loss: 0.5783 - accuracy: 0.7092 - val_loss: 0.5855 - val_accuracy: 0.6934
Epoch 7/10
153/153 [==============================] - 1s 5ms/step - loss: 0.5732 - accuracy: 0.7121 - val_loss: 0.5814 - val_accuracy: 0.7025
Epoch 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for RNN (assuming the data is time-series)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshaped = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Flatten the time-series data
X_train_flattened = np.squeeze(X_train_reshaped)
X_test_flattened = np.squeeze(X_test_reshaped)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)


# Initialize the CNN model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=64, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.23)

# Evaluate the CNN on the test set
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"CNN Accuracy: {accuracy:.2f}")
print("CNN Classification Report:\n", classification_report(y_test, y_pred))


Epoch 1/10
147/147 [==============================] - 4s 13ms/step - loss: 0.6289 - accuracy: 0.6598 - val_loss: 0.6179 - val_accuracy: 0.6664
Epoch 2/10
147/147 [==============================] - 1s 5ms/step - loss: 0.6107 - accuracy: 0.6732 - val_loss: 0.6109 - val_accuracy: 0.6643
Epoch 3/10
147/147 [==============================] - 1s 3ms/step - loss: 0.6049 - accuracy: 0.6762 - val_loss: 0.6103 - val_accuracy: 0.6728
Epoch 4/10
147/147 [==============================] - 1s 4ms/step - loss: 0.6020 - accuracy: 0.6796 - val_loss: 0.6048 - val_accuracy: 0.6771
Epoch 5/10
147/147 [==============================] - 1s 4ms/step - loss: 0.5980 - accuracy: 0.6915 - val_loss: 0.6086 - val_accuracy: 0.6835
Epoch 6/10
147/147 [==============================] - 1s 4ms/step - loss: 0.5967 - accuracy: 0.6877 - val_loss: 0.6087 - val_accuracy: 0.6778
Epoch 7/10
147/147 [==============================] - 0s 3ms/step - loss: 0.5938 - accuracy: 0.6877 - val_loss: 0.6091 - val_accuracy: 0.6800
Epoch